In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# *Deep Reinforced Learning*

Hoy en día el estado del arte en el reconocimiento de patrones está dominado por las **redes neuronales profundas**

- Visión Computacional: Redes Neuronales Convolucionales, Adversarios generativos
- Reconocimiento de habla: Redes Recurrentes, WaveNet
- Procesamiento de lenguaje Natural: Transformers

Las redes neuronales son excelentes para representar el mundo: modelos

> Podemos aprovechar esta capacidad para diseñar mejores algoritmos de aprendizaje reforzado

## Recuerdo: Value Learning

En este tipo de algoritmos usamos un política de máxima utilidad para escoger acciones

$$
\pi^*(s) = \text{arg} \max_{a\in \mathcal{A}} Q(s, a)
$$

Y el problema entonces se reduce a aprender **Q**, *e.g* Q-Learning

Sin embargo Q-learning tiene limitaciones: 
- requiere de heurísticas para explorar 
- espacio de acciones debe ser discreto
- espacio de estados debe ser discreto

De hecho el estado no puede ser demasiado grande, como veremos a continuación

### Ambiente: [Space invaders](https://es.wikipedia.org/wiki/Space_Invaders)

Originalmente un juego de Arcade lanzando en 1978, en 1980 tuvo una versión para ATARI 2600

> El objetivo es derrivar a los extraterrestres usando un cañon antes de que desciendan a la Tierra

- El cañon puede moverse a la izquierda, a la derecha y disparar
- Hay cuatro escudos con los que el cañon puede protegerse de los disparos enemigos
- Mientras menos enemigos más rápido se mueven

In [ ]:
import gym
from time import sleep

env = gym.make("SpaceInvaders-v0") 
env.reset()
end = False

while not end:
    a = env.action_space.sample()
    s, r, end, info = env.step(a)
    env.render() 
    sleep(.02)     

In [ ]:
env.close()

In [ ]:
# Espacio de estados
display(env.observation_space)
# Espacio de acciones
display(env.action_space)
display(env.action_space.n)
display(env.env.get_action_meanings())

Por ejemplo si usamos como estado un stack de 4 imágenes consecutivas y asumimos que cada pixel tiene 255 niveles:

$$
255^{210\cdot160\cdot3\cdot4}
$$

## Aproximación de funciones

Claramente, el espacio de estados del ejemplo anterior es imposible de mantener en una tabla Q

Ojo: este espacio de estados está aun lejos de un problema del mundo real

> Nos va a faltar memoría para guardar la tabla y datos para poder entrenar nuestro agente

¿Qué podemos hacer?

> Usar una representación más compacta para Q

En lugar de tener una tabla con todos las combinaciones estado/acción podemos

> Aproximar **Q** usando un **modelo paramétrico**

Esta es la idea principal tras *Value function approximation* (VFA) y *Q function approximation*

El caso más sencillo es usar un **modelo lineal en sus parámetros**

$$
\begin{align}
\hat Q_\theta(s,a) &= \theta_0 \phi_0(s, a) + \theta_1 \phi_1(s, a) + \theta_2 \phi_2(s, a) + \ldots + \theta_M \phi_M (s,a) \nonumber \\
&= \sum_{j=0}^M \theta_j \phi_j (s,a) \nonumber
\end{align}
$$

donde 
- $\{ \theta\}$ es un vector de parámetros con $M+1$ componentes
- $\{\phi\}$ es un conjunto de funciones base, *e.g.* polinomios, Fourier, árbol de decisión, kernels

En lugar de aprender $Q$ explicitamente el objetivo es aprender $\theta$

> La cantidad de parámetros es ahora independiente de la dimensionalidad del estado



Al igual que antes nuestro objetivo es acercanos a la solución de la Ecuación de Bellman

Podemos escribir esto como el siguiente problema de optimización

$$
\min_\theta \| R(s,a) + \gamma \max_{a' \in \mathcal{A}} \hat Q_\theta( s',a') - \hat Q_\theta(s,a)\|^2
$$

de donde podemos aprender $\theta$ iterativamente usando usando gradiente descendente 

$$
\theta_j \leftarrow \theta_j + 2 \alpha \left(R(s,a) + \gamma \max_{a' \in \mathcal{A}} \hat Q_\theta (s',a') - \hat Q_\theta(s,a) \right) \phi_j(s,a)
$$

Sin embargo, un modelo lineal podría ser muy limitado

En la unidad 1 estudiamos el estado del arte en aproximación de funciones: **redes neuronales artificiales**

> A continuación veremos como usar redes profundas para aproximar la función Q

## Deep Q-Network (DQN) 

> En [(Minh et al. 2013)](https://arxiv.org/abs/1312.5602) se usaron redes neuronales profundas de tipo convolucional para resolver una serie de juegos de ATARI con Aprendizaje Reforzardo obteniendo [desempeño sobre-humano en muchos de las pruebas](https://deepmind.com/blog/article/deep-reinforcement-learning). El modelo, llamado *Deep Q-network*, utiliza como estado el valor de todos los píxeles de cuatro cuadros consecutivos.

La idea clave es

> Aprovechar la capacidad de las redes neuronales profundas para representar datos complejos, e.g. imágenes 

o más en concreto

> Aproximar la función Q usando una red convolucional entrenada directamente sobre los píxeles

Veremos primero una formulación general y luego su aplicación al caso de imágenes

**Modelo**

- La entrada de la red es el estado $s$. 
    - El vector de estado puede tener valores continuos o discretos
- La salida de la red neuronal son los valores $Q(s, a_1), Q(s, a_2), \ldots, Q(s, a_N)$
    - Se considera un espacio de acciones discreto
    - Esto es más eficiente que considerar $a'$ como una entrada y retornar $Q(s,a')$
- La cantidad y tipo de las capas intermedias es decisión del usuario
    - Si tenemos datos continuos (atributos) usamos capas completamente conectadas
    - Si usamos píxeles es natural usar capas convolucionales
- La función de perdida que se ocupa en DQN es el error cuadrático medio entre la ecuación de Bellman y la predicción de la red

$$
\mathcal{L}(\theta) = \mathbb{E}\left[\left \| R(s,a) + \gamma \max_{a' \in \mathcal{A}} Q_\theta(s', a') - Q_\theta(s, a)\right \|^2\right]
$$

Recordemos: $s'$ es el estado al que llegamos luego de ejecutar $a$ sobre $s$

## DQN con estados continuos: El retorno del carro con péndulo

Previamente fue necesario discretizar el estado para construir la matriz Q

En DQN podemos obviar este paso y sus complicaciones

Vamos a usar una red neuronal con 3 capas completamente conectadas, 4 entradas y 2 salidas

In [ ]:
import torch

class MultilayerPerceptron(torch.nn.Module):    
    def __init__(self, n_input, n_output, n_hidden=10):
        super(type(self), self).__init__()
        self.hidden1 = torch.nn.Linear(n_input, n_hidden)
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
        self.output = torch.nn.Linear(n_hidden, n_output)
        self.activation = torch.nn.ReLU()
        
    def forward(self, x):
        h = self.activation(self.hidden1(x))
        h = self.activation(self.hidden2(h))
        return  self.output(h)

## Experience Replay y Memory replay

El entrenamiento usando sólo una instancia es bastante ruidoso (y un poco lento)

Consideremos también que estamos entrenando con muestras muy correlacionadas (no iid)

Sabemos que esto puede introducir sesgos el entrenamiento de una red neuronal

> Para entrenar la DQN adecuadamente  (Minh et al 2013) propone un astuto "truco" llamado *Experience replay*  

Esto consiste en almacenar la historia del agente en una memoria: *replay memory*

Cada elemento en la memoria es una tupla $(s_t, a_t, r_{t+1}, s_{t+1})$

Con esto se crean mini-batches en orden aleatorio para entrenar



In [ ]:
class ReplayMemory:
    
    def __init__(self, state_dim, memory_length=2000): 
        self.length = memory_length
        self.pointer = 0
        self.filled = False
        # Tensores vacíos para la historia
        self.s_current = torch.zeros((memory_length,) + state_dim)
        self.s_future = torch.zeros((memory_length,) + state_dim)
        self.a = torch.zeros(memory_length, 1, dtype=int)
        self.r = torch.zeros(memory_length, 1)
        # Adicionalmente guardaremos la condición de término
        self.end = torch.zeros(memory_length, 1, dtype=bool)
    
    def push(self, s_current, s_future, a, r, end):
        # Agregamos una tupla en la memoria
        self.s_current[self.pointer] = s_current
        self.s_future[self.pointer] = s_future
        self.a[self.pointer] = a
        self.r[self.pointer] = r 
        self.end[self.pointer] = end
        if self.pointer + 1 == self.length:
            self.filled = True
        self.pointer =  (self.pointer + 1) % self.length
        
    def sample(self, size=64):        
        # Extraemos una muestra aleatoria de la memoria
        if self.filled:
            idx = np.random.choice(self.length, size)
        elif self.pointer > size:
            idx = np.random.choice(self.pointer, size)
        else:
            return None        
        return self.s_current[idx], self.s_future[idx], self.a[idx], self.r[idx], self.end[idx]    
    

## *Double DQN*


En [(van Hasselt, Guer y Silver, 2015)](https://arxiv.org/pdf/1509.06461.pdf) los autores notaron un problema importante en DQN. 

> Cuando calculamos $Q(s, a)$ y $\max Q(s', a')$ usando la misma red es muy posible que sobre-estimemos la calidad de nuestro objetivo. Adicionalmente si el objetivo cambia constantemente el entrenamiento será inestable


La solución propuesta consiste en usar redes neuronales distintas para la escoger la acción y para calcular el objetivo (ecuación de Bellman)

$$
\mathcal{L}(\theta, \phi) = \mathbb{E}\left[\left \| R(s,a) + \gamma Q_\phi\left(s', \text{arg}\max_{a' \in \mathcal{A}} Q_\theta(s', a')\right) - Q_\theta(s, a)\right\|^2\right]
$$

Usamos $Q_\theta$ con parámetros $\theta$ para escoger la acción: *policy network*

Usamos $Q_\phi$ con parámetros $\phi$ para construir el objetivo: *target network*

- Ambas redes comparten arquitectura y cantidad de neuronas
- Sólo se optimiza la *policy network*
- Después de un cierto número de épocas los parametros de la policy network "se copian" en la *target network*

¿Cada cuantas épocas actualizo la *target network*? Otro hyperparámetro para el algoritmo... 

**Otras consideraciones**

- El optimizador a usar es ADAM: Gradiente descendente con tasa de aprendizaje adaptiva
- Se usa la heurística $\epsilon$ greedy para favorecer la exploración

In [ ]:
import torch
import copy
    
class DeepQNetwork:
    def __init__(self, q_model, gamma=0.99, double_dqn=False, learning_rate=1e-3, 
                 target_update_freq=500, clip_grads=True, clip_error=True):
        self.double_dqn = double_dqn
        self.gamma = gamma
        self.q_policy = q_model
        self.n_output = q_model.output.out_features
        self.clip_error = clip_error
        self.clip_grads = clip_grads
        self.update_counter = 0
        self.target_update_freq = target_update_freq
        
        self.criterion = torch.nn.MSELoss()
        self.optimizer = torch.optim.RMSprop(self.q_policy.parameters(), lr=learning_rate)
        
        if double_dqn:
            self.q_target = copy.deepcopy(self.q_policy)
            self.q_target.eval()
    
    def select_action(self, state, epsilon=0.):
        # Estrategia epsilon greedy para seleccionar acción
        if torch.rand(1).item() < 1. - epsilon: 
            self.q_policy.eval()
            with torch.no_grad():
                q = self.q_policy(state)[0]
                a = q.argmax().item()
                q = q[a]
            self.q_policy.train()
        else:
            q = None
            a = torch.randint(high=self.n_output, size=(1,)).item() 
        
        return a, q
    
    def update(self, mini_batch):
        self.update_counter += 1
        state, state_next, action, reward, end = mini_batch
        # Calcular Q
        q_current = self.q_policy(state).gather(1, action)
        with torch.no_grad():
            if not self.double_dqn:
                q_next_best = self.q_policy(state_next).max(1, keepdim=True)[0]
            else:
                action_next = self.q_policy(state_next).argmax(dim=1, keepdim=True)
                q_next_best = self.q_target(state_next).gather(1, action_next)               
        # Construir el target: r + gamma*max Q(s', a')
        td_target = reward
        td_target[~end] += self.gamma*q_next_best[~end]
        td_target[end] = -1.
        # Calcular pérdido y sus gradientes
        self.optimizer.zero_grad()
        loss = self.criterion(q_current, td_target)
        if self.clip_error:
            loss.clamp_(-1., 1.)
        loss.backward()
        # Cortar gradientes grandes (mejora la estabilidad)
        if self.clip_grads:
            for param in self.q_policy.parameters():
                param.grad.data.clamp_(-1., 1.)
            # torch.nn.utils.clip_grad.clip_grad_norm_(self.model.parameters(), 10)
        # Actualizar
        self.optimizer.step()
        # Transfer policy to target
        self.transfer_policy2target()
        # Retornar el valor de la loss
        return loss.item()
    
    def transfer_policy2target(self):
        if self.double_dqn:            
            if self.update_counter % self.target_update_freq == 0:
                self.q_target.load_state_dict(self.q_policy.state_dict())
                return True
        return False

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.signal import convolve

fig, ax = plt.subplots(4, figsize=(6, 5), sharex=True, tight_layout=True)

def smooth_data(x, window_length=10):
    return convolve(x, np.ones(window_length)/window_length, mode='valid')

def update_plot(step, episode, smooth_window=10, target=195, target_update=500):
    for ax_ in ax:
        ax_.cla()
    episodes = np.arange((episode))
    ax[0].scatter(episodes, diagnostics['rewards'], s=1)      
    if episode > smooth_window:
        ax[0].plot(episodes[:-smooth_window+1], 
                   smooth_data(diagnostics['rewards']), alpha=0.5, lw=2)        
    ax[1].plot(episodes, diagnostics['loss'])
    ax[2].plot(episodes, np.array(diagnostics['q_sum'])/(np.array(diagnostics['q_N'])+1e-4))
    #for update_time in diagnostics['target_updates']:
    #    ax[1].plot([update_time, update_time], 
    #               [diagnostics['loss'].min(), diagnostics['loss'].max()], 'k--', alpha=0.1)                  
                   
    ax[0].plot(episodes, [target]*len(episodes), 'k--')
    ax[0].set_ylabel('Recompensa');
    ax[1].set_ylabel('Loss')
    ax[2].set_ylabel('Q promedio')
    ax[3].plot(episodes, epsilon(episodes))
    ax[3].set_ylabel('Epsilon')
    ax[3].set_xlabel('Episodios')
    ax[0].set_title("Paso %d" % (step))
    fig.canvas.draw()

## Experimentos

Usemos la clase DeepQNetwork para entrenar una agente que resuelva el problema del carro con péndulo

- Si uso `memory_length=1` es equivalente a no usar *replay memory*. Mientras más corta es la memoria más ruidoso es el entrenamiento. 
- Con el parámetro `double_dqn` podemos activar la red objetivo que se actualizará cada `target_update_freq` pasos
- Otros parámetros a considerar son `gamma`, el tamaño del batch, la tasa de aprendizaje y la arquitectura de la red

In [ ]:
import numpy as np
import gym
from tqdm.notebook import tqdm

torch.manual_seed(123)

env = gym.make("CartPole-v0")
n_state = (env.observation_space.shape[0],) # Número de estados
n_action = env.action_space.n # Número de acciones

dqn_model = DeepQNetwork(q_model=MultilayerPerceptron(n_state[0], n_action, n_hidden=16),
                         gamma = 0.99,
                         double_dqn=True,
                         target_update_freq=100,
                         learning_rate=1e-4)

def epsilon(episode, epsilon_init=1., epsilon_end=0.1, epsilon_rate=1e-2):
    return epsilon_end + (epsilon_init - epsilon_end) * np.exp(-epsilon_rate*episode) 

memory = ReplayMemory(n_state, memory_length=100000)        

diagnostics = {'rewards': [0], 'loss': [0],
               'q_sum': [0], 'q_N': [0]}

episode = 1
end = False
state = env.reset()
for step in tqdm(range(30000)):    
    # Escoger acción
    state = torch.tensor(state).float()
    a, q = dqn_model.select_action(state.unsqueeze(0), epsilon(episode))
    if q is not None:
        diagnostics['q_sum'][-1] += q
        diagnostics['q_N'][-1] += 1
    # Ejecutar acción
    s, r, end, info = env.step(a)        
    # Guardar en memoria
    memory.push(state, torch.tensor(s).float(), a, torch.tensor(r/200.), end)
    # Actualizar modelo    
    mini_batch = memory.sample(32)
    if not mini_batch is None:
        diagnostics['loss'][-1] += dqn_model.update(mini_batch)            
    # Guardar nuevo estado y continuar
    diagnostics['rewards'][-1] += r
    state = s 
    # Preparar siguiente episodio
    if end:
        if episode % 25 == 0:
            update_plot(step, episode)
        episode += 1  
        end = False
        state = env.reset()
        diagnostics['rewards'].append(0)
        diagnostics['loss'].append(0)
        diagnostics['q_sum'].append(0)
        diagnostics['q_N'].append(0)

**Intuición**

> La *loss* está subiendo, mi algoritmo está mal

No necesariamente, recordemos que el "training set" está cambiando con cada paso

Es normal ver que los *loss* tenga subidas abruptas 

En cierto modo estas subidas son deseables pues significa que nuestro algoritmo está "sorprendiendose" con nuevos estados

> La recompensa promedio se fue en picada, mi algoritmo está mal

Lo que está ocurriendo es un problema común en deep RL que se llama: **"olvido catastrófico"** 

El modelo es cada vez más bueno resolviendo el problema y la memoria se llena de "buenas transiciones"

Los *minibatches* ya no tienen ejemplos erroneos: El modelo ya no sabe lo que está mal 

Se puede combatir con *prioritized replay memory* o con otros heurísticas

- Reservar un espacio en la memoria para las transiciones (de exploración) iniciales 
- **Early stopping:** Guardar el modelo de mayor recompensa y detener el aprendizaje si la recompensa no sube en un cierto número de episodios

**Probando el agente DQN**

In [ ]:
import numpy as np
import gym

env = gym.make("CartPole-v0")

In [ ]:
env.reset()

for k in range(500):
    env.render()
    state = torch.from_numpy(np.array(env.state).astype('float32')).unsqueeze(0)
    a, q = dqn_model.select_action(state)
    s_future, r, end, info = env.step(a)
    if r == 0:
        display(k)
        break
display(end, k)

In [ ]:
env.close()

## [Rainbow DQN](https://arxiv.org/pdf/1710.02298.pdf)

Rainbow DQN es un trabajo que reune todos los avances relacionados a DQN

1. *Double DQN*: Usar un modelo separado para calcular los targets
1. *Replay memory*: Almacenar experiencias pasadas y muestrear aleatoriamente de la memoria para disminuir la correlación 
1. [*Prioritized experience replay*](https://arxiv.org/abs/1511.05952): El minibatch se construye ponderando las experiencias anteriores en función del error. Experiencias con más error tienen más probabilidad de ser escogidas. "Se puede aprender más de los ejemplos más difíciles"
1. [*Dueling networks*](https://arxiv.org/abs/1511.06581): Se separa la función $Q(s,a) = V(s) + A(s,a)$, donde cada función es una red neuronal que comparten capas. Con esto se puede aprender el valor de cada estado de forma desacoplada a la importancia relativa de las acciones
1. [*Noisy nets*](https://arxiv.org/abs/1706.10295): Capas de redes neuronales que incluyen un ruido paramétrico en sus parámetros lo que permite explorar sin necesidad de la heurística $\epsilon$-greddy
1. entre otros